# Convert the hourly netCDF ragged array to parquet

In [38]:
import numpy as np
import xarray as xr
import awkward as ak

In [39]:
path_gdp = '../data/process/gdp_v2.00.nc'
ds = xr.open_dataset(path_gdp)

In [96]:
ds

<xarray.Dataset>
Dimensions:                (traj: 17324, obs: 165754333)
Coordinates:
    ID                     (traj) int64 ...
    longitude              (obs) float32 -17.74 -17.75 -17.74 ... -30.32 -30.37
    latitude               (obs) float32 ...
    time                   (obs) datetime64[ns] ...
    ids                    (obs) int64 ...
Dimensions without coordinates: traj, obs
Data variables: (12/54)
    rowsize                (traj) int64 417 2005 1970 1307 ... 4572 17097 3127
    location_type          (traj) bool ...
    WMO                    (traj) int32 ...
    expno                  (traj) int32 ...
    deploy_date            (traj) datetime64[ns] ...
    deploy_lon             (traj) float32 ...
    ...                     ...
    err_sst                (obs) float32 ...
    err_sst1               (obs) float32 ...
    err_sst2               (obs) float32 ...
    flg_sst                (obs) int8 ...
    flg_sst1               (obs) int8 ...
    flg_sst2               (obs) int8 ...
Attributes: (12/15)
    title:             Global Drifter Program hourly drifting buoy collection
    history:           Version 2.00.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2022-02-02T15:57:04.179158
    publisher_name:    GDP Drifter DAC
    publisher_email:   aoml.dftr@noaa.gov
    ...                ...
    metadata_link:     https://www.aoml.noaa.gov/phod/dac/dirall.html
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2022) to be submitted. Elipot et al. (2...
    summary:           Global Drifter Program hourly data

In [197]:
offset = ak.layout.Index32(np.insert(np.cumsum(ds.rowsize), 0, 0))

longitude = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.longitude))
latitude = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.latitude))
time = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.time.values))
ids = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.ids))
ve = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.ve))
vn = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.vn))
gap = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.gap))
err_lat = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.err_lat))
err_lon = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.err_lon))
err_ve = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.err_ve))
err_vn = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.err_vn))
drogue_status = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.drogue_status))
sst = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.sst))
sst1 = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.sst1))
sst2 = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.sst2))
err_sst = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.err_sst))
err_sst1 = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.err_sst1))
err_sst2 = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.err_sst2))
flg_sst = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.flg_sst))
flg_sst1 = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.flg_sst1))
flg_sst2 = ak.layout.ListOffsetArray32(offset, ak.layout.NumpyArray(ds.flg_sst2))

In [287]:
obs = ak.Array(
        ak.layout.RecordArray(
            [
                longitude,
                latitude,
                #time,
                ids,
                ve,
                vn,
                #gap,
                err_lat,
                err_lon,
                err_ve,
                err_vn,
                drogue_status,
                sst,
                sst1,
                sst2,
                err_sst,
                err_sst1,
                err_sst2,
                flg_sst,
                flg_sst1,
                flg_sst2,
            ],
            [
                "longitude",
                "latitude",
                #"time",
                "ids",
                "ve",
                "vn",
                #"gap",
                "err_lat",
                "err_lon",
                "err_ve",
                "err_vn",
                "drogue_status",
                "sst",
                "sst1",
                "sst2",
                "err_sst",
                "err_sst1",
                "err_sst2",
                "flg_sst",
                "flg_sst1",
                "flg_sst2",
            ],
        )
    )

In [265]:
array = ak.Array(
        ak.layout.RecordArray(
            [
                ak.layout.NumpyArray(ds.ID),
                ak.layout.NumpyArray(ds.rowsize),
                ak.layout.NumpyArray(ds.location_type),
                ak.layout.NumpyArray(ds.WMO),                
                ak.layout.NumpyArray(ds.expno),               
                ak.layout.NumpyArray(ds.deploy_date.values),
                ak.layout.NumpyArray(ds.deploy_lat),
                ak.layout.NumpyArray(ds.deploy_lon),
                ak.layout.NumpyArray(ds.end_date.values),
                ak.layout.NumpyArray(ds.end_lat),
                ak.layout.NumpyArray(ds.end_lon),
                ak.layout.NumpyArray(ds.drogue_lost_date.values),
                #ak.layout.NumpyArray(ds.type_death),
                #ak.layout.NumpyArray(ds.type_buoy),
                #ak.layout.NumpyArray(ds.DeploymentShip),
                #ak.layout.NumpyArray(ds.DeploymentStatus),
                #ak.layout.NumpyArray(ds.BuoyTypeManufacturer),
                #ak.layout.NumpyArray(ds.BuoyTypeSensorArray),
                #ak.layout.NumpyArray(ds.CurrentProgram),
                #ak.layout.NumpyArray(ds.PurchaserFunding),
                #ak.layout.NumpyArray(ds.SensorUpgrade),
                #ak.layout.NumpyArray(ds.Transmissions),
                #ak.layout.NumpyArray(ds.DeployingCountry),
                #ak.layout.NumpyArray(ds.DeploymentComments),
                #ak.layout.NumpyArray(ds.ManufactureYear),
                #ak.layout.NumpyArray(ds.ManufactureMonth),
                #ak.layout.NumpyArray(ds.ManufactureSensorType),
                #ak.layout.NumpyArray(ds.ManufactureVoltage),
                #ak.layout.NumpyArray(ds.FloatDiameter),
                #ak.layout.NumpyArray(ds.SubsfcFloatPresence),
                #ak.layout.NumpyArray(ds.DrogueType),
                #ak.layout.NumpyArray(ds.DrogueLength),
                #ak.layout.NumpyArray(ds.DrogueBallast),
                #ak.layout.NumpyArray(ds.DragAreaAboveDrogue),
                #ak.layout.NumpyArray(ds.DragAreaOfDrogue),
                #ak.layout.NumpyArray(ds.DragAreaRatio),
                #ak.layout.NumpyArray(ds.DrogueCenterDepth),
                #ak.layout.NumpyArray(ds.DrogueDetectSensor),
                obs.layout,
            ],
            [
                "rowsize",
                "ID",
                "location_type",
                "WMO",
                "expno",
                "deploy_date",
                "deploy_lat",
                "deploy_lon",
                "end_date",
                "end_lat",
                "end_lon",
                "drogue_lost_date",
                #"type_death",
                #"type_buoy",
                #"DeploymentShip",
                #"DeploymentStatus",
                #"BuoyTypeManufacturer",
                #"BuoyTypeSensorArray",
                #"CurrentProgram",
                #"PurchaserFunding",
                #"SensorUpgrade",
                #"Transmissions",
                #"DeployingCountry",
                #"DeploymentComments",
                #"ManufactureYear",
                #"ManufactureMonth",
                #"ManufactureSensorType",
                #"ManufactureVoltage",
                #"FloatDiameter",
                #"SubsfcFloatPresence",
                #"DrogueType",
                #"DrogueLength",
                #"DrogueBallast",
                #"DragAreaAboveDrogue",
                #"DragAreaOfDrogue",
                #"DragAreaRatio",
                #"DrogueCenterDepth",
                #"DrogueDetectSensor",
                "obs",
            ],
        )
    )

In [276]:
typebuoy = ak.layout.ListOffsetArray32(
    ak.layout.Index32(np.cumsum([0] + [len(x) for x in ds.type_buoy.values], dtype=np.int32)),
    ak.layout.NumpyArray(
        np.frombuffer(b"".join(ds.type_buoy.values), dtype="u1"),
        parameters={"__array__": "char"},
    ),
    parameters={"__array__": "string"},
)

In [ ]:
ds.type_buoy[4500]

<xarray.DataArray 'type_buoy' ()>
array(b'SVPB   ', dtype='|S15')
Coordinates:
    ID       int64 60950920
Attributes:
    long_name:  Buoy type (see https://www.aoml.noaa.gov/phod/dac/dirall.html)
    units:      -

In [288]:
gdp_parquet = 'process/gdp_v2.00.parquet'

ak.to_parquet(
    obs,
    gdp_parquet,
    use_dictionary=["type_buoy", "DeploymentShip", 
                    "DeploymentStatus", "BuoyTypeManufacturer", 
                    "BuoyTypeSensorArray", "Transmissions", 
                    "PurchaserFunding", "SensorUpgrade"
                    "DeployingCountry", "DeploymentComments", 
                    "ManufactureSensorType"],
    compression="zstd",
    compression_level=9,
    use_byte_stream_split=[
        #"deploy_date",
        #"deploy_lat",
        #"deploy_lon",
        #"end_date",
        #"end_lat",
        #"end_lon",
        #"drogue_lost_date",
        "obs.longitude",
        "obs.latitude",
        #"obs.time",
        "obs.ve",
        "obs.vn",
        "obs.err_lat",
        "obs.err_lon",
        "obs.err_ve",
        "obs.err_vn",
        #"obs.gap",
        "obs.sst",
        "obs.sst1",
        "obs.sst2",
        "obs.err_sst",
        "obs.err_sst1",
        "obs.err_sst2",
        "obs.flg_sst",
        "obs.flg_sst1",
        "obs.flg_sst2",
    ],
)